## Sigmoid
$ f(x) = \frac{1}{1 + e^-x} $  

</br> 
sigmoid의 미분값은 sigmoid(1-sigmoid)로 0.5가 제일 큰 값이다.  0.5보다 작은 값을 가지는 미분값이 뒤로 갈 수록 지속해서 곱해지면, vanishing problem이 생긴다.

## ReLU

$ f(x) = max(0, x) $

## Softmax

In [3]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [10]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [8]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [11]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

# Initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.1953, -0.4404,  0.0139,  ...,  0.7510,  1.5190,  0.9637],
        [ 0.4146, -1.0660, -0.9969,  ..., -0.3259, -0.2695, -1.3316],
        [ 0.2600, -1.1821,  0.0065,  ...,  1.0648, -1.2251,  0.9841],
        ...,
        [ 0.0279, -1.7242, -1.1723,  ...,  1.3651, -0.0689, -0.7931],
        [ 0.0952,  0.1939,  0.1636,  ..., -1.0489,  0.6224, -1.7181],
        [ 2.6504,  0.6297, -1.6571,  ..., -0.0781, -1.1398, -0.8711]],
       requires_grad=True)

In [12]:
# define cost/Loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)  #softmax is internally computed
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [15]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
    
    for X, Y in data_loader:
        #reshape input image into [batch_size by 784]
        # Label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        avg_cost += cost /total_batch
        
    print("Epoch: ", "%04d" % (epoch+1), "cost = ", "{:.9f}".format(avg_cost))

print('Learning finished')

Epoch:  0001 cost =  16.194974899
Epoch:  0002 cost =  16.194980621
Epoch:  0003 cost =  16.194957733
Epoch:  0004 cost =  16.194952011
Epoch:  0005 cost =  16.194990158
Epoch:  0006 cost =  16.194950104
Epoch:  0007 cost =  16.194952011
Epoch:  0008 cost =  16.194950104
Epoch:  0009 cost =  16.194944382
Epoch:  0010 cost =  16.194972992
Epoch:  0011 cost =  16.194980621
Epoch:  0012 cost =  16.194982529
Epoch:  0013 cost =  16.194923401
Epoch:  0014 cost =  16.194969177
Epoch:  0015 cost =  16.194948196
Learning finished


## mnist_nn

In [16]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [17]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.3778,  0.4159, -0.1067,  ...,  0.6471,  1.0248, -0.4888],
        [ 2.0965, -0.1699,  0.3719,  ...,  0.3424,  0.4114,  1.5675],
        [ 0.7887,  0.4940, -0.3835,  ..., -0.7595, -0.3343,  1.6731],
        ...,
        [ 0.5905, -0.1444,  0.3107,  ..., -0.7852,  1.2604, -0.7070],
        [-1.1099, -0.1722,  0.7348,  ..., -2.2059, -0.1641,  0.2070],
        [-2.5434,  0.4488, -1.9866,  ...,  1.7574,  0.1124,  1.2071]],
       requires_grad=True)

In [18]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [19]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 61.090724945
Epoch: 0002 cost = 14.688576698
Epoch: 0003 cost = 7.568543911
Epoch: 0004 cost = 4.955924511
Epoch: 0005 cost = 3.416701794
Epoch: 0006 cost = 2.396533251
Epoch: 0007 cost = 1.918075204
Epoch: 0008 cost = 1.531555414
Epoch: 0009 cost = 1.419201374
Epoch: 0010 cost = 1.117504835
Epoch: 0011 cost = 1.069085479
Epoch: 0012 cost = 1.016389847
Epoch: 0013 cost = 0.927181780
Epoch: 0014 cost = 0.812819600
Epoch: 0015 cost = 0.831833363
Learning finished
